<a href="https://colab.research.google.com/github/yotambs/Line_Detector/blob/main/Scratch_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Installing libraries that do no`t exsit in google colab")
!pip install segmentation_models 


Installing libraries that do no`t exsit in google colab
     |████████████████████████████████| 50 kB 6.2 MB/s 


In [3]:
import segmentation_models as sm 
import random
import numpy as np
from tensorflow import keras
import tensorflow.keras.callbacks as callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

Segmentation Models: using `keras` framework.


# New Section

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preprocessing(X):
  return np.uint8(X/np.max(X))

sm.set_framework('tf.keras')
datagen = ImageDataGenerator(preprocessing_function=preprocessing)
#datagen = ImageDataGenerator()
img_height = 512
img_width = 512
EPOCHS = 150 

#############################################
#  100 images data 
#############################################
#train_imgs_data_dir  =  './drive/MyDrive/Colab Notebooks/scratch_detector/Data/train'
#train_masks_data_dir =  './drive/MyDrive/Colab Notebooks/scratch_detector/Data/masks'#/home/yotam/workSpace/Line_detection/Data/masks"
###############################################
#  10 images data for quick train and overfit #
###############################################
#train_imgs_data_dir  =  '/content/drive/MyDrive/Colab Notebooks/scratch_detector/Data/input_0'
#train_masks_data_dir =  '/content/drive/MyDrive/Colab Notebooks/scratch_detector/Data/masks_0'
#data_sz = 10
###############################################
#
###############################################
train_imgs_data_dir  =  './drive/MyDrive/Colab Notebooks/scratch_detector/Data/input'
train_masks_data_dir =  './drive/MyDrive/Colab Notebooks/scratch_detector/Data/GT'
data_sz = 990

seed = 42
batch_Sz = 8
keras.backend.set_image_data_format('channels_last')

BACKBONE = 'resnet18'
preprocess_input = sm.get_preprocessing(BACKBONE)

image_generator = datagen.flow_from_directory(
    directory= train_imgs_data_dir,
    seed = seed,
    color_mode='rgb',
    target_size=(img_height, img_width),
    batch_size=batch_Sz,
    class_mode= None)

mask_generator = datagen.flow_from_directory(
    directory= train_masks_data_dir,
    seed = seed,
    target_size=(img_height, img_width),
    batch_size=batch_Sz,
    class_mode= None)

train_generator = zip(image_generator,mask_generator)
val_generator   = zip(image_generator,mask_generator)


model = sm.Unet(BACKBONE, activation='relu')
#####################################
#   1
#####################################
model.compile(
     'Adam',
     loss=sm.losses.dice_loss, #loss=sm.losses.bce_jaccard_loss,
     metrics=[sm.metrics.iou_score],
)
#####################################
#   2
#####################################
#model.compile(
#    'Adam',
#    'binary_crossentropy',
#     ['binary_accuracy']
#)
#####################################
#   3
#####################################
#model.compile(
#     'Adam',
#     loss=sm.losses.bce_jaccard_loss, #loss=sm.losses.bce_jaccard_loss,
#     metrics=[sm.metrics.iou_score],
# )


model.summary()
callbacks_list=[callbacks.ReduceLROnPlateau(monitor='loss',factor=0.75,patience=1,verbose = 1,min_lr=1e-5),
callbacks.ModelCheckpoint(filepath='./Scratch_detector_binary_focal.h5',monitor='loss',save_best_only=True,verbose = 1)]


loss = model.fit(
    train_generator,
    steps_per_epoch= data_sz/batch_Sz,
    epochs = EPOCHS,
    validation_data=val_generator,
    validation_steps = 2,
    callbacks=callbacks_list)

Found 990 images belonging to 1 classes.
Found 990 images belonging to 1 classes.
44933120/44920640 [==============================] - 0s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None

In [ ]:
import pandas as pd

model_history = pd.DataFrame(loss.history)
model_history['epoch'] = loss.epoch

fig, ax = plt.subplots(1, figsize=(8,6))
num_epochs = model_history.shape[0]

ax.plot(np.arange(0, num_epochs), model_history["iou_score"], 
        label="Training iou_score")
ax.plot(np.arange(0, num_epochs), model_history["val_iou_score"], 
        label="Validation iou_score")
ax.plot(np.arange(0, num_epochs), model_history["loss"], 
        label="Training loss")
ax.plot(np.arange(0, num_epochs), model_history["val_loss"], 
        label="Validation loss")
ax.legend()
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
#model_history

In [ ]:
from tensorflow import keras
from google.colab.patches import cv2_imshow
import numpy as np 
import cv2 
######################################
# load model
###################################### 
#model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/scratch_detector/Scratch_detector_bce.h5')
#model = keras.models.load_model('/content/Scratch_detector_bce.h5')
#print(f"model input:: {model.input.shape}")
######################################
# load image 
######################################
#img = cv2.imread('/content/drive/MyDrive/Data/input/5.png', cv2.IMREAD_COLOR)
#img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/scratch_detector/Data/input_0/images/5.png', cv2.IMREAD_COLOR)
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/scratch_detector/Data/test/3.png', cv2.IMREAD_COLOR)
cv2_imshow(img)
print(img.shape)
######################################
# inferrence 
######################################
img = np.expand_dims(img,axis=0)
print(img.shape)
output = model.predict(img)
print(f"model predicition:: {output.shape}")
output = np.squeeze(output)
norm_output = (output/np.max(output))*255
cv2_imshow(norm_output)

# New Section

In [ ]:
print(np.max(output))
print(np.min(output))